<a href="https://colab.research.google.com/github/eduardo-reinert/furb-pln/blob/main/Unidade_2/PLN_unidade2_atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLN: Unidade 2 – Coleta e Processamento de Dados Textuais

**Alunos**: Augusto Arraga, Eduardo Reinert, Vinícius Silva

> Iremos fazer o nosso trabalho em cima de **reviews dos jogos mais vendidos** nas últimas semanas, vindas da plataforma *Steam*.
> <br/><br/>Para obtenção dos dados, utilizamos as seguintes chamadas de endpoints de APIs:
> - Resgate dos jogos mais vendidos dos últimos tempos (1° passo):
>   - https://store.steampowered.com/search/results/?filter=topsellers&json=1&page=2
> - Resgate dos detalhes específicos de um jogo
>   - https://store.steampowered.com/api/appdetails?appids=730
> - Resgate das reviews do jogo:
>   - https://store.steampowered.com/appreviews/730?json=1&num_per_page=500&language=brazilian


## **Base de dados textuais**

#### **Justificativa e explicação da escolha**

Optamos por trabalhar com **reviews de jogos mais vendidos na plataforma Steam** porque essas avaliações representam opiniões reais de usuários, permitindo capturar sentimentos, percepções e padrões de linguagem natural sobre os jogos.

A escolha da Steam se justifica por três fatores principais (não necessariamente em ordem de importância):

1. **Acesso à informação estruturada via API** — a plataforma disponibiliza endpoints confiáveis para resgatar listas de jogos, detalhes específicos e reviews de usuários.

2. **Relevância e atualidade** — ao focar nos jogos mais vendidos nas últimas semanas, garantimos que os dados reflitam tendências recentes de mercado e comportamento de usuários.

3. **Diversidade de conteúdo** — as reviews variam em extensão, linguagem, complexidade e estilo, oferecendo um material rico para tarefas de Processamento de Linguagem Natural (PLN), como análise de sentimentos, tokenização, lematização e classificação de texto.

#### **Adequação dos dados às tarefas de PLN (volume, abrangência, variedade)**

A base de dados obtida apresenta características adequadas para aplicações de PLN:

* **Volume**: Cada jogo resgatado através da API, pode retornar um total de **50 reviews**, oferecendo uma quantidade suficiente de texto para análise estatística e treinamento de modelos de PLN. Levando em conta que, ao acessar a primeira e segunda página da lista de jogos mais vendidos do momento nos retorna 50 jogos, podemos obter um volume de até **2500 reviews por execução**.

* **Abrangência**: A coleta engloba diversos títulos, gêneros e estilos de jogos, garantindo uma cobertura representativa de diferentes tipos de conteúdo textual. Esta abrangência é alcançada devido a coleta dinâmica de jogos através da **lista de jogos mais vendidos do momento**, que é atualizada diariamente com novos lançamentos, jogos populares em promoção, etc.

* **Variedade**: As reviews incluem textos curtos e longos, linguagem formal e informal, elogios e críticas, bem como uso de diferentes estruturas gramaticais, o que é essencial para testar técnicas de tokenização, normalização, remoção de ruídos, stemming e lematização.

#### **Organização e interpretabilidade da base de dados**
A base de dados foi estruturada de forma a facilitar análise e interpretação:

* Cada jogo é representado como um objeto com informações essenciais (após processamento, que você irá conferir nos códigos em sequência): ``steam_appid``, ``name``, ``total_positive``, ``total_negative`` e ``reviews``.

* As reviews são objetos aninhados contendo o texto original, identificadores e metadados do autor, garantindo rastreabilidade.

* Para análise de PLN, adicionamos campos processados (`tokens`, `stemmed`, `lemmatized`) que permitem comparar o texto antes e depois do pré-processamento.

Essa organização hierárquica e padronizada facilita a extração de insights, aplicação de técnicas de NLP e construção de modelos de análise de sentimentos ou classificação de texto.


## **Scraping ou acesso via API (código)**

Conforme dito no início desse arquivo Jupyter/Colab:

> <br/><br/>Para obtenção dos dados, utilizamos as seguintes chamadas de endpoints de APIs:
> - Resgate dos jogos mais vendidos dos últimos tempos (1° passo):
>   - https://store.steampowered.com/search/results/?filter=topsellers&json=1&page=2
> - Resgate dos detalhes específicos de um jogo
>   - https://store.steampowered.com/api/appdetails?appids=730
> - Resgate das reviews do jogo:
>   - https://store.steampowered.com/appreviews/730?json=1&num_per_page=500&language=brazilian

## **Passo-a-passo da obtenção das reviews de jogos na Steam via API**

#### Bibliotecas utilizadas

In [1]:
#Bibliotecas necessárias
!pip install requests
import requests

!pip install -U spacy unidecode

!python -m spacy download pt_core_news_sm

import nltk
nltk.download('stopwords')
nltk.download('punkt')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 48.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### **1° Passo -> Resgatar os 50 jogos mais vendidos dos últimos tempos**

Resgatamos os IDs dos 50 jogos mais vendidos (chamados de 'app_id') para utilizar nas APIs posteriores

In [2]:
#Resgate da 1a e 2a lista de jogos mais vendidos (1a lista dos mais vendidos, e em sequência a 2a lista)
import re
import requests

# API endpoints
api_url1 = "https://store.steampowered.com/search/results/?filter=topsellers&json=1&page=1"
api_url2 = "https://store.steampowered.com/search/results/?filter=topsellers&json=1&page=2"

# Get responses
primeira_lista = requests.get(api_url1).json()
segunda_lista = requests.get(api_url2).json()

# Junta o resultado de ambas as listas
all_items = primeira_lista["items"] + segunda_lista["items"]

# Extrai os app_ids para uso posterior
app_ids = []
for item in all_items:
    match = re.search(r"apps/(\d+)/", item["logo"])
    if match:
        app_ids.append(match.group(1))

print(app_ids)


['1285190', '730', '1675200', '2357570', '3354750', '3008130', '553850', '1172470', '3180070', '2767030', '1030300', '3354820', '3472040', '3115220', '1172710', '2444750', '3230400', '2807960', '3354800', '2694490', '1085660', '2688950', '1984270', '381210', '1222670', '236390', '230410', '367520', '2686630', '2073620', '3902620', '2947440', '3159330', '3527290', '2073850', '3224770', '3293260', '2486820', '3513350', '252490', '3405690', '2928600', '359550', '1151340', '1086940', '1304930', '3164500', '1599340', '3240220', '582660']


### **2° Passo: Resgatar os detalhes de cada jogo específico**
### **3° Passo: Resgatar as reviews de cada jogo**

In [3]:
# Lista final combinada
combined_data = []

for app_id in app_ids:
    detalhes_url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    reviews_url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&language=brazilian"

    detalhes_response = requests.get(detalhes_url).json()
    reviews_response = requests.get(reviews_url).json()

    # Extrair dados do detalhes_response
    detalhes_data = detalhes_response.get(app_id, {}).get("data", {})
    steam_appid = detalhes_data.get("steam_appid")
    name = detalhes_data.get("name")

    # Extrair dados do reviews_response
    query_summary = reviews_response.get("query_summary", {})
    total_reviews = query_summary.get("num_reviews")
    total_positive = query_summary.get("total_positive")
    total_negative = query_summary.get("total_negative")
    reviews = reviews_response.get("reviews", [])

    # Monta JSON final
    combined_data.append({
        "steam_appid": steam_appid,
        "name": name,
        "total_reviews": total_reviews,
        "total_positive": total_positive,
        "total_negative": total_negative,
        "reviews": reviews
    })

# Lista JSON
print(combined_data)


[{'steam_appid': 1285190, 'name': 'Borderlands 4', 'total_reviews': 20, 'total_positive': 60, 'total_negative': 51, 'reviews': [{'recommendationid': '204107444', 'author': {'steamid': '76561198048366131', 'num_games_owned': 87, 'num_reviews': 6, 'playtime_forever': 272, 'playtime_last_two_weeks': 272, 'playtime_at_review': 42, 'last_played': 1757763762}, 'language': 'brazilian', 'review': 'Eu tenho computador acima do recomendado e o jogo toda insuportavelmente mal. Extremamente mal otimizado. Vou pedir reembolso.', 'timestamp_created': 1757622538, 'timestamp_updated': 1757622538, 'voted_up': False, 'votes_up': 89, 'votes_funny': 6, 'weighted_vote_score': '0.872324645519256592', 'comment_count': 3, 'steam_purchase': True, 'received_for_free': False, 'written_during_early_access': False, 'primarily_steam_deck': False}, {'recommendationid': '204196476', 'author': {'steamid': '76561199437182145', 'num_games_owned': 0, 'num_reviews': 25, 'playtime_forever': 89, 'playtime_last_two_weeks': 8

## **Limpeza e preparação dos dados**

### **Criação de novo JSON com apenas os dados necessários**

Executando o código acima, obtivemos a lista ``combined_data``, onde cada elemento do JSON está relacionado a um dos 50 jogos resgatados da lista de mais vendidos dos últimos tempos.

Cada jogo retorna um objeto JSON com alguns metadados relacionados ao jogo, e as reviews (avaliações). Porém, iremos filtrar esta lista JSON criada e resgatar apenas os dados que nos interessam.

Exemplo de como irá ficar a nova lista de dados filtrada:
```json
[
  {
    "steam_appid": 1285190,
    "name": "Borderlands 4",
    "total_positive": 0,
    "total_negative": 0,
    "reviews": []
  },
  {
    "steam_appid": 1085660,
    "name": "Destiny 2",
    "total_positive": 2526,
    "total_negative": 345,
    "reviews": [
      {
        "recommendationid": "202137083",
        "review": "...",         
        "voted_up": True/False    
      },
      ...
    ]
  },
  ...
]

```

In [5]:
import json
filtered_data = []

for game in combined_data:
    filtered_reviews = [
        {
            "recommendationid": r.get("recommendationid"),
            "review": r.get("review"),
            "voted_up": r.get("voted_up")
        }
        for r in game.get("reviews", [])
    ]

    filtered_game = {
        "steam_appid": game.get("steam_appid"),
        "name": game.get("name"),
        "total_positive": game.get("total_positive"),
        "total_negative": game.get("total_negative"),
        "reviews": filtered_reviews
    }

    filtered_data.append(filtered_game)

print(json.dumps(filtered_data, indent=2, ensure_ascii=False))


[
  {
    "steam_appid": 1285190,
    "name": "Borderlands 4",
    "total_positive": 60,
    "total_negative": 51,
    "reviews": [
      {
        "recommendationid": "204107444",
        "review": "Eu tenho computador acima do recomendado e o jogo toda insuportavelmente mal. Extremamente mal otimizado. Vou pedir reembolso.",
        "voted_up": false
      },
      {
        "recommendationid": "204196476",
        "review": "junto com Wuchang que lançou um patch fake pra enganar bobos sobre otimização, veio borderlands com um guia de como otimizar seu pc pra jogar ( ligar muletas e deixar no low ) parece até piada, serio.\n\npor mais que eu seja fã e esteja louca pra jogar, acho que virou uma chave em mim e espero que vire essa chave em todos consequentemente, não dar mais dinheiro pra empresinha que não quer fazer o minimo pro jogo dela ser jogado, ''OTIMIZAR'' \n\n",
        "voted_up": false
      },
      {
        "recommendationid": "204300208",
        "review": "Cansei.\r\n\

### **Pré-processamento de texto para PLN em português**

O objetivo é **limpar e preparar os dados textuais** para análises posteriores (classificação, mineração de sentimentos, modelagem de tópicos, etc.).

1. **Remoção de ruídos**

   * Eliminação de tags HTML, links e caracteres especiais desnecessários usando expressões regulares (`re`).
   * Remove elementos que não trazem significado semântico para o texto.

2. **Normalização**

   * Conversão para minúsculas (`lowercase`).
   * Remoção de acentos usando `unidecode`.

3. **Tokenização**

   * Quebra do texto em unidades menores (tokens), geralmente palavras.
   * Usamos **spaCy** para tokenizar texto em português de forma confiável, evitando problemas com NLTK.

4. **Remoção de Stopwords e Pontuação**

   * Stopwords são palavras muito frequentes que não carregam significado importante (ex: "de", "a", "o").
   * Também removemos sinais de pontuação usando `string.punctuation` e a lista de stopwords do NLTK.

5. **Stemming**

   * Reduz cada palavra à sua raiz (radical), removendo sufixos.
   * Utilizamos o `SnowballStemmer` do NLTK com suporte ao português.

6. **Lematização**

   * Reduz as palavras à sua forma canônica (lematizada), considerando contexto e gramática.
   * Usamos o modelo `pt_core_news_sm` do **spaCy** para português.

---

#### Bibliotecas utilizadas

| Biblioteca | Função                                                  |
| ---------- | ------------------------------------------------------- |
| spaCy      | Tokenização e lematização em português                  |
| NLTK       | Stopwords e stemming (SnowballStemmer)                  |
| unidecode  | Normalização e remoção de acentos                       |
| re         | Expressões regulares (remoção de tags HTML, URLs, etc.) |
| string     | Auxiliar para remover pontuação                         |

---

#### Objetivo Final

* Enriquecer cada `review` do nosso dataset com campos processados contendo:

  * Tokens
  * Stems (radicais)
  * Lemas


#### Código

In [6]:
import spacy
import re
import string
import copy
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Carregar spaCy português
nlp = spacy.load("pt_core_news_sm")

# Stopwords e stemmer
stop_words = set(stopwords.words('portuguese'))
stemmer = SnowballStemmer('portuguese')

review_before = []
review_after = []

for game in filtered_data:
    for review in game['reviews']:
        review_before.append(copy.deepcopy(review))

        text = review.get('review', '')

        if text:
            # Limpeza
            text = re.sub(r'<.*?>', ' ', text)
            text = re.sub(r'http\S+|www\.\S+', ' ', text)
            text = unidecode(text.lower())

            # Tokenização com spaCy
            doc = nlp(text)
            tokens = [token.text for token in doc if token.text not in string.punctuation and token.text not in stop_words]

            # Stemming (NLTK)
            stemmed = [stemmer.stem(t) for t in tokens]

            # Lematização (spaCy)
            lemmatized = [token.lemma_ for token in nlp(" ".join(tokens))]

            review['processed'] = {
                "tokens": tokens,
                "stemmed": stemmed,
                "lemmatized": lemmatized
            }
        else:
            review['processed'] = {
                "tokens": [],
                "stemmed": [],
                "lemmatized": []
            }

        review_after.append(copy.deepcopy(review))


## **Apresentação dos resultados após processamento**

### Reviews (antes e após processamento)

In [7]:
for before, after in zip(review_before, review_after):
    print("=== Review Original ===")
    print(before.get('review', ''))
    print("--- Review Processado ---")
    processed = after.get('processed', {})
    print("Tokens:", processed.get('tokens', []))
    print("Stemmed:", processed.get('stemmed', []))
    print("Lematized:", processed.get('lemmatized', []))
    print("\n" + "="*50 + "\n")


=== Review Original ===
Eu tenho computador acima do recomendado e o jogo toda insuportavelmente mal. Extremamente mal otimizado. Vou pedir reembolso.
--- Review Processado ---
Tokens: ['computador', 'acima', 'recomendado', 'jogo', 'toda', 'insuportavelmente', 'mal', 'extremamente', 'mal', 'otimizado', 'vou', 'pedir', 'reembolso']
Stemmed: ['comput', 'acim', 'recomend', 'jog', 'tod', 'insuport', 'mal', 'extrem', 'mal', 'otimiz', 'vou', 'ped', 'reembols']
Lematized: ['computador', 'acima', 'recomendar', 'jogo', 'toda', 'insuportavelmente', 'mal', 'extremamente', 'mal', 'otimizar', 'ir', 'pedir', 'reembolso']


=== Review Original ===
junto com Wuchang que lançou um patch fake pra enganar bobos sobre otimização, veio borderlands com um guia de como otimizar seu pc pra jogar ( ligar muletas e deixar no low ) parece até piada, serio.

por mais que eu seja fã e esteja louca pra jogar, acho que virou uma chave em mim e espero que vire essa chave em todos consequentemente, não dar mais dinhei

## **"Bonus Round" (informações extras)**

### Automatização na coleta (atualização periódica)

Conforme descrito nos tópicos anteriores, cada vez que o código é executado, é resgatado dados da Steam via API de seus jogos mais vendidos dos últimos tempos. A cada minuto esta lista é atualizada na plataforma oficial da Steam, o que torna esse Colab atualizado conforme o momento do qual você está o executando.

Além disso, toda a coleta é feita automaticamente (é necessário apenas inicializar a execução do código e todo o processo é feito posteriormente).